In [ ]:
import numpy as np

In [ ]:
class Item:
    def __init__(self, price, weight):
        self.price = price
        self.weight = weight

In [ ]:
file_path = "./debug_10.txt"

with open(file_path, "r") as file:
    content = file.readlines()


first_line = content[0].strip().split()

number_of_items = int(first_line[0])
knapsack_capacity = int(first_line[1])

items = []

class Config:
    def __init__(self,
                 number_of_items,
                 knapsack_capacity) -> None:
        self.number_of_items = number_of_items
        self.knapsack_capacity = knapsack_capacity
        self.weight_surpass_punish_coefficient = 2
        self.size_of_population = 600
        self.number_of_iterations = 120
        self.mutation_rate_coefficient = 0.00
        
config = Config(number_of_items, knapsack_capacity)

In [ ]:


print("Number of items is: ", config.number_of_items)
print("Capacity:           ", config.knapsack_capacity)

# parse file into list of custom Item objects
for line in content[1:]:
    numbers = line.strip().split()
    item_cost, item_weight = map(int, numbers)
    items.append(Item(item_cost, item_weight))

assert len(items) == number_of_items

for item in items:
    print(item.price, item.weight)

In [ ]:
class Individual:
    def __init__(self, item_count):
        
        # array([0, 1, 0, 1, 1])
        self.mask = np.random.choice([0, 1], size=(item_count,))

    def get_fitness(self, items, knapsack_capacity):
        weight = np.dot(self.mask, [item.weight for item in items])
        cost = np.dot(self.mask, [item.price for item in items])

        # print("Weight: ", weight, "Cost: ", cost)

        weight_surpassed_by = max(weight - knapsack_capacity, 0)
        punishment = (weight_surpassed_by * config.weight_surpass_punish_coefficient)
        fitness = max(cost - punishment, 0)

        # print((cost,weight,punishment, fitness)) # debug
        return fitness
    
    def mutate(self):
        for i in range(len(self.mask)):
            if np.random.rand() < (1/ len(self.mask)) * config.mutation_rate_coefficient:
                self.mask[i] = 1 - self.mask[i]

    def get_cost(self, items):
        return np.dot(self.mask, [item.price for item in items])


In [ ]:
def selection(population):
    fitness_scores = [individual.get_fitness(items, knapsack_capacity) for individual in population]
    total_fitness = sum(fitness_scores)
    probabilities = [fitness / total_fitness for fitness in fitness_scores]
    selected_individuals = np.random.choice(population, size=len(population)*2, p=probabilities)
    return selected_individuals

def get_children(mating_pool):
    children = []
    for i in range(0, len(mating_pool), 2):
        parent1 = mating_pool[i]
        parent2 = mating_pool[i+1]

        split_point = np.random.randint(0, number_of_items)
        child = Individual(number_of_items)
        child.mask = np.concatenate((parent1.mask[:split_point], parent2.mask[split_point:]))
        
        
        child.mutate()
        
        children.append(child)
    return children


# Generate random population
population = [Individual(number_of_items) for _ in range(config.size_of_population)]

average = []
max_acheived_fitness = []

for i in range(config.number_of_iterations):

    pop_costs = [individual.get_cost(items) for individual in population]
    average.append(np.mean(pop_costs))
    max_acheived_fitness.append(np.max(average))
    
    mating_pool = selection(population)

    children = get_children(mating_pool)

    population = children

import matplotlib.pyplot as plt

plt.plot(average)
plt.plot(max_acheived_fitness)
plt.xlabel('Iteration')
plt.ylabel('Fitness')
plt.title('Max knapsack cost in Time')
plt.legend(['Average generation Fitness', 'Max Achieved Fitness'])
plt.show()

print(max_acheived_fitness[-1])


In [ ]:
cost = 295
weight = config.knapsack_capacity
weight_surpassed_by = max(weight - knapsack_capacity, 0)

fitness =  max(cost - (weight_surpassed_by * config.weight_surpass_punish_coefficient), 0)
fitness